In [67]:
import pandas as pd 
import os 
import plotly.express as px
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
geoLoc = Nominatim(user_agent="GetLoc")

# import pandas-profiling


In [23]:
df = pd.read_csv("/home/mohammed/Downloads/nb.csv") 
dl = pd.read_csv("/home/mohammed/Downloads/driver_locations_during_request.csv")

In [24]:
df.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [25]:
db = df.dropna()

In [26]:
db['Trip End Time'] = pd.to_datetime(db['Trip End Time'])
db['Trip Start Time'] = pd.to_datetime(db['Trip Start Time'])
db['trip_time']= (db['Trip End Time']- db['Trip Start Time']).dt.total_seconds()
db['trip_time']= (db['trip_time']/60).round(3)
db.head()

/tmp/ipykernel_13689/803530810.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13689/803530810.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13689/803530810.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_1368

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,trip_time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,1.550
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,29.400
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,41.350
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,13.583
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.617


In [27]:
# import pandas_profiling
# # data_fifa = pd.read_csv('fifa.csv')
# profile = db.profile_report(sensitive = True,title='Pandas Profiling Report',samples=None)
# profile.to_file(output_file="fifa_pandas_profiling.html")

In [28]:
dx = db 
dr = dx
dr=(dx.groupby('Trip Origin',as_index=False).agg(Count=('Trip ID', 'count'))).sort_values(by="Count",ascending=False).reset_index(drop=True)

# trip_origin = df['Trip Origin'].groupby(df['Trip Origin'])
# trip_origin.count().sort_values(ascending =False)
init_df = pd.merge(dr, db, how= "inner", left_on = 'Trip Origin', right_on = 'Trip Origin')
# init_df.head()
dr.head()

,Trip Origin,Count
0,"6.6137933,3.357997",865
1,"6.462727,3.55679",668
2,"6.4293597,3.4334968",576
3,"6.4286308,3.5155123",544
4,"6.622069799999999,3.383825",538


In [29]:
location_df = dr
location_df[['ori_Latitude', 'ori_Longitude']] = location_df['Trip Origin'].str.strip('( )').str.split(',', expand=True)
# location_df[['des_Latitude', 'des_Longitude']] = location_df['Trip Destination'].str.strip('( )').str.split(',', expand=True)
# location_df = location_df.drop(columns  =['Trip Origin','Trip Destination'])
location_df[['ori_Latitude', 'ori_Longitude']] = location_df[['ori_Latitude', 'ori_Longitude']].apply(pd.to_numeric)
location_df.head()

,Trip Origin,Count,ori_Latitude,ori_Longitude
0,"6.6137933,3.357997",865,6.613793,3.357997
1,"6.462727,3.55679",668,6.462727,3.556790
2,"6.4293597,3.4334968",576,6.429360,3.433497
3,"6.4286308,3.5155123",544,6.428631,3.515512
4,"6.622069799999999,3.383825",538,6.622070,3.383825


In [30]:


fig = px.scatter_mapbox(location_df.head(1000), lat="ori_Latitude", lon="ori_Longitude", hover_name="Count",color="Count", hover_data=location_df.columns.tolist(),
                        size = 'Count',zoom=3, height=600)
fig.update_layout(mapbox_style="stamen-terrain")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# import plotly.express as px
# px.set_mapbox_access_token(open("/home/mohammed/Desktop/10 Academy projects/token.mapbox_token").read())
# # df = px.data.carshare()
# fig = px.scatter_mapbox(location_df.head(100), lat="ori_Latitude", lon="ori_Longitude", hover_data = location_df.columns.tolist(),size_max='Count', zoom=10)
#                 #   color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
# fig.show()

Destination locations

In [35]:
ds=(db.head().groupby('Trip Destination',as_index=False).agg(Count=('Trip ID', 'count'))).sort_values(by="Count",ascending=False).reset_index(drop=True)
ds[['des_Latitude', 'des_Longitude']] = ds['Trip Destination'].str.strip('( )').str.split(',', expand=True).apply(pd.to_numeric)
ds.dtypes


Trip Destination     object
Count                 int64
des_Latitude        float64
des_Longitude       float64
dtype: object

In [37]:
fig_ds = px.scatter_mapbox(ds.head(1000), lat="des_Latitude", lon="des_Longitude", hover_name="Count",color="Count", hover_data=ds.columns.tolist(),
                        size = 'Count',zoom=3, height=600)
fig_ds.update_layout(mapbox_style="stamen-terrain")
fig_ds.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_ds.show()

In [61]:
ds_top = ds.head(100)
ds_top["destination_location_name"] = ds_top.apply(lambda x:str(geoLoc.reverse(str(x["des_Latitude"])+","+str(x["des_Longitude"]))),axis=1)


/tmp/ipykernel_13689/2631045711.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [72]:
ds_top.head()

,Trip Destination,Count,des_Latitude,des_Longitude,destination_location_name
0,"6.4567308,3.5309404",844,6.456731,3.530940,"Ojomu, Eti Osa, Lagos, 105102, Nigeria"
1,"6.448969300000001,3.5231794",793,6.448969,3.523179,"Ojomu, Eti Osa, Lagos, 105102, Nigeria"
2,"6.5837939,3.333551899999999",732,6.583794,3.333552,"MMA2 New Domestic Terminal, Airport Road, Ikej..."
3,"6.452345299999999,3.527539",727,6.452345,3.527539,"Bourdillon Court Estate, Lekki, Ojomu, Eti Osa..."
4,"6.497595,3.383098",718,6.497595,3.383098,"Yaba, Lagos Mainland, Lagos, 101212, Nigeria"


In [89]:
def split(text:str,ind:int=-4):
    try:
        text=text.split(',')[ind]
        return text 
    except Exception:
        if ind > 1:
            print("Error occured")
        else:
            ind+=1
            return split(text,ind)
ds_top['destination_location_name']= ds_top['destination_location_name'].str.strip('( )').str.split(',', expand=True)

/tmp/ipykernel_13689/230464017.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
ds_top.head()

,Trip Destination,Count,des_Latitude,des_Longitude,destination_location_name
0,"6.4567308,3.5309404",844,6.456731,3.530940,None
1,"6.448969300000001,3.5231794",793,6.448969,3.523179,None
2,"6.5837939,3.333551899999999",732,6.583794,3.333552,None
3,"6.452345299999999,3.527539",727,6.452345,3.527539,None
4,"6.497595,3.383098",718,6.497595,3.383098,None


In [82]:
# lable = ds_top["destination_location_name"].value_counts()
# size = (ds_top["destination_location_name"].value_counts())
ds_top["destination_location_name"]=ds_top['destination_location_name'].apply(lambda x:x.split())
top_locations=ds_top.groupby('destination_location_name',as_index=False).agg({"Count":"sum"}).sort_values(by="Count",ascending=False)
# plt.bar(size,labels= lable,hight)
top_locations.set_index('destination_location_name',inplace=True)
top_locations.plot(kind = 'bar')
plt.show()

AttributeError: 'list' object has no attribute 'split'

In [44]:
db[['ori_Latitude', 'ori_Longitude']] = db['Trip Origin'].str.strip('( )').str.split(',', expand=True).apply(pd.to_numeric)
db[['des_Latitude', 'des_Longitude']] = db['Trip Destination'].str.strip('( )').str.split(',', expand=True).apply(pd.to_numeric)
db.head()

/tmp/ipykernel_13689/3487543445.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13689/3487543445.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13689/3487543445.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_1

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,trip_time,ori_Latitude,ori_Longitude,des_Latitude,des_Longitude
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,1.550,6.508813,3.377403,6.650970,3.345031
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,29.400,6.431671,3.455538,6.428081,3.472189
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,41.350,6.631679,3.338898,6.508324,3.359040
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,13.583,6.572757,3.367708,6.584881,3.361407
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.617,6.601042,3.276634,6.450107,3.391615


In [45]:
geoLoc = Nominatim(user_agent="GetLoc")

db["origin_location_name"] = db.apply(lambda x:str(geoLoc.reverse(str(x["ori_Latitude"])+","+str(x["ori_Longitude"]))),axis=1)
db["destination_location_name"] = db.apply(lambda x:str(geoLoc.reverse(str(x["des_Latitude"])+","+str(x["des_Longitude"]))),axis=1)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=6.5508295&lon=3.3991093&format=json&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fb76f7e4af0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

Data set 2


In [39]:
dl.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN


In [44]:
full_df = pd.merge(db, dl, how= "inner", left_on = 'Trip ID', right_on = 'order_id')
full_df.tail()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,trip_time,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
1555713,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557736,517948,243774,rejected,6.469036,3.566877,NaN,NaN
1555714,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557737,517948,245447,rejected,6.425431,3.527358,NaN,NaN
1555715,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557738,517948,239866,rejected,6.440013,3.525378,NaN,NaN
1555716,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557739,517948,243774,rejected,6.469036,3.566877,NaN,NaN
1555717,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557740,517948,244170,rejected,6.469390,3.572079,NaN,NaN


In [56]:
total = full_df.isnull().sum().sort_values(ascending=False)
percent = (full_df.isnull().sum()/full_df.isnull().count()).sort_values(ascending=False)
missing_full_df = pd.concat([total, percent], axis=1, keys=['Total', 'Missing Percent'])
missing_full_df['Missing Percent'] = missing_full_df['Missing Percent'].apply(lambda x: x * 100)
missing_full_df.loc[missing_full_df['Missing Percent'] > 10][:10]

,Total,Missing Percent
created_at,1555718,100.0
updated_at,1555718,100.0


In [63]:
full_df.drop(columns = ['created_at','updated_at'])

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,trip_time,id,order_id,driver_id,driver_action,lat,lng
0,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.617,1,392001,243828,accepted,6.602207,3.270465
1,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.617,2,392001,243588,rejected,6.592097,3.287445
2,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.617,3,392001,243830,rejected,6.596133,3.281784
3,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.617,4,392001,243539,rejected,6.596142,3.280526
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.617,5,392001,171653,rejected,6.609232,3.288800
...,...,...,...,...,...,...,...,...,...,...,...,...
1555713,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557736,517948,243774,rejected,6.469036,3.566877
1555714,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557737,517948,245447,rejected,6.425431,3.527358
1555715,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557738,517948,239866,rejected,6.440013,3.525378
1555716,517948,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,102.783,1557739,517948,243774,rejected,6.469036,3.566877
